### Tbot - Trade Bot

 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 
### Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this api. 

In [1]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import hmac
import time
import datetime
import hashlib
import binascii

### API token, secret key and acess keys
API tokens (and other sensitive information) shall be stored in a secrets.py file that doesn't get pushed to your local Git repository. The AcessKey and SecretKey are provided by the server admin.

In [2]:
from secrets import SecretKey
from secrets import AccessKey
from novadax import RequestClient as NovaClient


nova_client = NovaClient(AccessKey, SecretKey)

### Importing tht List of currencies, making the API Call and the table Dataframe
List of crypto-currencies, get balance, and table creation

In [3]:
balance = nova_client.get_account_balance()
bdata = balance['data']

bcolumns = ['Currency', 'Balance']
bdataframe = pd.DataFrame(columns = bcolumns,)
bs = 0
bcr = {}

for bsymbol in bdata:
    pdis = bdata[bs]
    bis = pdis['balance']
    bic = pdis['currency']
    bcr[f"{bic}"] = f"{bis}"   
    bdataframe.append(
        pd.Series([
            bic,
            bis,
        ], index = bcolumns),
        ignore_index = True)
    bdataframe = bdataframe.append(
    pd.Series([
        bic,
        bis,
    ], index=bcolumns),
        ignore_index=True)
    bs = bs + 1  

bdataframe.sort_values('Balance', ascending = False, inplace = True)
bdataframe.reset_index(drop = True, inplace = True)
bdataframe

,Currency,Balance
0,BTT,96.09424
1,USDT,0.09682579
2,DGB,0.0353
3,LINK,0.0088601
4,DOGE,0.00881
5,XLM,0.00447
6,NULS,0.0044
7,IOTA,0.003905
8,XRP,0.0036
9,XTZ,0.000097


### Symbol position, Dolar price and timestamp
For the execution to be done independently for each symbol, the below cell is needed:

In [4]:
symbol = 'ETH_USDT'
symbols = ['ETH_USDT']
currency = 'R$'
my_columns = ['Symbol', 'Currency Price', 'Number of Shares to Buy']
e_dataframe = pd.DataFrame(columns = my_columns)
for symbol in symbols:
    r = 0
    ldolar = nova_client.list_trades(symbol, limit=10)
    for i in ldolar['data']:  
        fddata = nova_client.list_trades(symbol, limit=10)
        fdata = fddata['data']
        fdt = fdata[r]
        price = fdt['price']
        amount = fdt['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        shares_to_buy = (float(edolarwal)/float(price))
        etsp = (fdt['timestamp'])
        r = r + 1
        for i in fddata['data'][:1]:
            e_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        shares_to_buy
                    ]
                    ,index = my_columns),
                ignore_index=True)
    
    
            e_dataframe = e_dataframe.append(
                                                pd.Series([
                                                    symbol, 
                                                    price,
                                                    shares_to_buy
                                                    ],
                                                    index = my_columns
                                                    ),
                                                ignore_index = True)

e_dataframe

,Symbol,Currency Price,Number of Shares to Buy
0,ETH_USDT,1589.45,0.000061
1,ETH_USDT,1592.75,0.000061
2,ETH_USDT,1592.88,0.000061
3,ETH_USDT,1599.16,0.000061
4,ETH_USDT,1591.35,0.000061
5,ETH_USDT,1582.52,0.000061
6,ETH_USDT,1600,0.000061
7,ETH_USDT,1606.7,0.000060
8,ETH_USDT,1606.75,0.000060
9,ETH_USDT,1626.61,0.000060


### Creating the momentum table
For mommentum we use the last order minus the order from an amount x of time. The place from wich the data comes is main importance on the formula. As stated in the comment section on the first versions, data must come from the common filled orders or the last trades from the users history, here the second case is uded, as well from this version on, but the first ones used common filled orders.

In [5]:
listtrades = nova_client.list_orders(symbol, 'FILLED', limit=5)
ldata = listtrades['data']
lindice= ldata[0]
amount = lindice['amount']
my_columns = ['Symbol', 'Amount', 'Number of Shares to Buy','Last Orders Price', 'B/S', 'Momentum Price',  'momentum-raw-data']
m_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listtrades = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    for i in listtrades['data'][:9]:
        ldata = listtrades['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = ldc['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        #ref:        https://stackoverflow.com/questions/41383787/round-down-to-2-decimal-in-python
        sharestobuy = float(amount) - (float(edolarwal))/float(price)
        bs = ldc['side']
        order = float(amount) - float(sharestobuy)
        srto = float(order)
        
        #M = V - Vx ; M representa o indicador de momentum, V o valor de fechamento atual e V(x_diasatras) o valor anterior
        #nao é deste modo que se utiliza o simbolo x na multiplicação. Vx seriam multiplos valores de V em x dias. Para este caso, Vi
        #veja que por conotaçao convencional parece nao fazer diferença, porém, na matemática, tais convenções são necessárias e de certo modo indispensáveis no tempo t
        #h = 
        vxdata = e_dataframe['Currency Price']
        vxi = vxdata[s]
        vx = vxi
        momentum = float(price) - float(vx)
        tsp = (ldc['timestamp'])
        s = s + 1
        m_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        sharestobuy,
                        price,
                        bs,
                        vx,
                        momentum,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        m_dataframe = m_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        sharestobuy,
                        price,
                        bs,
                        vx,
                        momentum,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

m_dataframe.sort_values('Last Orders Price', ascending = True, inplace = True)
m_dataframe.reset_index(drop = True, inplace = True)
m_dataframe

,Symbol,Amount,Number of Shares to Buy,Last Orders Price,B/S,Momentum Price,momentum-raw-data
0,ETH_USDT,0.0188,0.018750,1928.42,BUY,1582.52,345.90
1,ETH_USDT,0.0685,0.068450,1953.10,BUY,1599.16,353.94
2,ETH_USDT,0.05,0.049950,1953.10,BUY,1591.35,361.75
3,ETH_USDT,0.0186,0.018550,1953.26,SELL,1606.7,346.56
4,ETH_USDT,0.0186,0.018550,1953.42,SELL,1600,353.42
5,ETH_USDT,0.0186,0.018551,1967.68,SELL,1606.75,360.93
6,ETH_USDT,0.0675,0.067451,1983.00,BUY,1592.88,390.12
7,ETH_USDT,0.0674,0.067352,1999.85,SELL,1589.45,410.40
8,ETH_USDT,0.0674,0.067352,2034.00,SELL,1592.75,441.25


### Sending the requisition to the web
The content in the order requisition should be sent to the server following the instructions given by the company or individual that holds the coin. Here as columns the last used titles shall be all used in one go to give a more complete dataframe in our xlsx database.

In [6]:
#sort price [ok]
#wallet_amount [ok]
#dataframe momentum[ok]pt2[ok]
#buy lower(wallet_amount - amount)   [ok]  
#sell + profit(constant*market momentum)[ok]
#if sell = true; [buy lower < sell][(sell - (contant*momentum)/2 or (buy + constant/momentum)/2 [ok]
#P = S - B; SUM(Pi to Pn)  = Pi{(S - D) or (S - B)}
#in the if statement one more contidional is needed at the "or" statment. If or else(momentum>=x or else)
#20210222#v0-9#import all the list itens from the last dataframes and send them to the xlsx file. 
#20200225#v0-9-1# the data must come from the common filled orders or the last trades from the users history, 
#                 here the second case is uded, as well from this version on, first ones used common filled orders

In [7]:
#orders
ttrades = nova_client.list_trades(symbol, limit=9)
tdata = ttrades['data']
my_columns = ['Symbol', 'Amount', 'Last Orders Price', 'B/S', 'Momentum Price',  'momentum-raw-data', 'Status', 'Timestamp']
o_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listorders = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    for i in listorders['data'][:9]:
        ldata = listorders['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = float(ldc['amount'])
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        bs = ldc['side']
        
        #rework into a precise formula with momentum
        vxdata = tdata[s]
        vxi = vxdata['price']
        momentumprice = float(vxi)
        momentumraw = float(price) - float(vxi)
        
        #https://stackoverflow.com/questions/45140034/python-convert-seconds-to-datetime-date-and-time
        from datetime import datetime
        #datetime.fromtimestamp(tspi).strftime("%A, %B %d, %Y %I:%M:%S")
        #https://docs.python.org/3/library/datetime.html
        tspi = float(ldc['timestamp']/1000)
        tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
        status = ldc['status']
        
        s = s + 1
        
        o_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        o_dataframe = o_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

o_dataframe.sort_values('Timestamp', ascending = False, inplace = True)
o_dataframe.reset_index(drop = True, inplace = True)
o_dataframe

,Symbol,Amount,Last Orders Price,B/S,Momentum Price,momentum-raw-data,Status,Timestamp
0,ETH_USDT,0.0674,1999.85,SELL,1589.45,410.40,CANCELED,20210221 13:56:38
1,ETH_USDT,0.0674,2034.00,SELL,1592.75,441.25,CANCELED,20210220 16:08:38
2,ETH_USDT,0.0675,1983.00,BUY,1592.88,390.12,FILLED,20210220 10:51:38
3,ETH_USDT,0.0685,1953.10,BUY,1599.16,353.94,CANCELED,20210219 18:38:12
4,ETH_USDT,0.0500,1953.10,BUY,1591.35,361.75,CANCELED,20210219 18:37:37
5,ETH_USDT,0.0188,1928.42,BUY,1582.52,345.90,CANCELED,20210219 15:15:20
6,ETH_USDT,0.0186,1953.42,SELL,1600.00,353.42,FILLED,20210219 15:14:51
7,ETH_USDT,0.0186,1953.26,SELL,1606.70,346.56,CANCELED,20210219 15:14:19
8,ETH_USDT,0.0186,1967.68,SELL,1606.75,360.93,CANCELED,20210219 15:13:35


In [8]:
#buy lower(wallet_amount - amount)  [ok]   
#sell + profit(constant*market momentum)[ok]


#if sell = true; [buy lower < sell][(sell - (contant*momentum)/2 or (buy + constant/momentum)/2
#P = S - B; SUM(Pi to Pn)  = Pi{(S - D) or (S - B)}

#in the if statement one more contidional is needed at the "or" statment. If or else(momentum>=x or else)
#20210222#v0-9#import all the list itens from the last dataframes and send them to the xlsx file. 
#20200225#v0-9-1# the data must come from the common filled orders or the last trades from the users history, 
#                 here the second case is uded, as well from this version on, first ones used common filled orders


if momentumprice > price :
    sellprofit = math.floor(price + price/100*1/2)/1.00
    result = nova_client.create_order(symbol, 'LIMIT', 'SELL', price = profit, amount = amount)  

#first block to be solved
if momentumprice < price and amount < 0.02:
    print("Market is not ready to sell")

if momentumprice < price and amount > 0.02:
    result = nova_client.create_order(symbol, 'LIMIT', 'BUY', price = bmprice, amount = amount)
    

Market is not ready to sell


In [24]:
#sub-account ballance
subId = 'n4'

blc = nova_client.subs_balance(subId)
blcdata = blc['data']

blccolumns = ['Currency', 'Balance']
blcdataframe = pd.DataFrame(columns = bcolumns,)
blcs = 0
blccr = {}

for bsymbol in blcdata:
    pdis = blcdata[blcs]
    bis = pdis['balance']
    bic = pdis['currency']
    blccr[f"{bic}"] = f"{bis}"   
    blcdataframe.append(
        pd.Series([
            bic,
            bis,
        ], index = bclccolumns),
        ignore_index = True)
    blcdataframe = blcdataframe.append(
    pd.Series([
        bic,
        bis,
    ], index=blccolumns),
        ignore_index=True)
    blcs = blcs + 1  

blcdataframe.sort_values('Balance', ascending = False, inplace = True)
blcdataframe.reset_index(drop = True, inplace = True)
blcdataframe

RequestException: ('A10007', 'Account not exist')

In [ ]:
#https://stackoverflow.com/questions/45140034/python-convert-seconds-to-datetime-date-and-time
from datetime import datetime
#datetime.fromtimestamp(tspi).strftime("%A, %B %d, %Y %I:%M:%S")
#https://docs.python.org/3/library/datetime.html
datetime.fromtimestamp(tspi).strftime("%Y %H:%M:%S")

### Format and export xlsx file
In the last section of the tbot a xlsx file is saved with the order table for later reading purposes.

In [ ]:
writer = pd.ExcelWriter('orders.xlsx', engine = 'xlsxwriter')
o_dataframe.to_excel(writer, sheet_name = 'orderlist', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
    'A':['Symbol', string_template],
    'B':['Amount', float_template],
    'C':['Price', dollar_template],
}

for column in column_formats.keys():
    writer.sheets['orderlist'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['orderlist'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

In [ ]:
writer.save()